# Explainability Playbook
This notebook demonstrates how the new `src/explainability` utilities support
Shapley-value style attributions, Integrated Gradients, and attention
rollout-based visualisation. Each section mirrors the TypeScript API with a
small NumPy/PyTorch-free prototype so that the conceptual steps remain clear.

## 1. SHAP via permutation sampling
We approximate Shapley values by randomly permuting features and accumulating
marginal contributions. The code below mirrors
`estimateShapValues` from `src/explainability/shap.ts`.

In [ ]:
import math
from typing import Callable, List
import random

def logistic_model(weights: List[float], bias: float) -> Callable[[List[float]], float]:
    def _forward(x: List[float]) -> float:
        z = sum(w * xi for w, xi in zip(weights, x)) + bias
        return 1.0 / (1.0 + math.exp(-z))
    return _forward

def shap_permutation(model: Callable[[List[float]], float],
                     x: List[float],
                     baseline: List[float],
                     permutations: int = 64,
                     seed: int = 7) -> List[float]:
    rng = random.Random(seed)
    contrib = [0.0 for _ in x]
    for _ in range(permutations):
        order = list(range(len(x)))
        rng.shuffle(order)
        current = baseline.copy()
        prev = model(current)
        for idx in order:
            current[idx] = x[idx]
            nxt = model(current)
            contrib[idx] += nxt - prev
            prev = nxt
    return [c / permutations for c in contrib]

model = logistic_model([1.4, -0.8], bias=-0.2)
x = [0.9, -1.1]
baseline = [0.0, 0.0]
shap_vals = shap_permutation(model, x, baseline, permutations=128)
prediction = model(x)
baseline_pred = model(baseline)
total_contrib = sum(shap_vals)
prediction, baseline_pred, total_contrib


## 2. Integrated Gradients along a straight path
Integrated Gradients requires access to the model's gradient. We craft an
analytic gradient for the logistic regression above and numerically integrate
along the path between a baseline and the target input.

In [ ]:
def logistic_gradient(weights: List[float], bias: float) -> Callable[[List[float]], List[float]]:
    def _grad(x: List[float]) -> List[float]:
        z = sum(w * xi for w, xi in zip(weights, x)) + bias
        p = 1.0 / (1.0 + math.exp(-z))
        return [w * p * (1 - p) for w in weights]
    return _grad

def integrated_gradients(grad_fn: Callable[[List[float]], List[float]],
                          baseline: List[float],
                          input_x: List[float],
                          steps: int = 50) -> List[float]:
    acc = [0.0 for _ in input_x]
    for step in range(1, steps + 1):
        alpha = step / steps
        point = [b + alpha * (xi - b) for b, xi in zip(baseline, input_x)]
        grad = grad_fn(point)
        for i, g in enumerate(grad):
            acc[i] += g
    return [(xi - b) * acc_i / steps for (xi, b), acc_i in zip(zip(input_x, baseline), acc)]

grad_fn = logistic_gradient([1.4, -0.8], bias=-0.2)
ig_vals = integrated_gradients(grad_fn, baseline, x, steps=100)
ig_vals, sum(ig_vals)


## 3. Attention rollout
We combine a stack of attention matrices (averaged over heads) using matrix
multiplication. Residual connections correspond to injecting an identity
matrix before multiplying.

In [ ]:
def attention_rollout(layers, add_identity=True):
    size = len(layers[0])
    def _normalize(matrix):
        return [[value / max(1e-9, sum(row)) for value in row] for row in matrix]
    result = [[1 if i == j else 0 for j in range(size)] for i in range(size)] if add_identity else layers[0]
    if add_identity:
        result = multiply(result, _normalize(layers[0]))
        start = 1
    else:
        result = _normalize(result)
        start = 1
    for layer in layers[start:]:
        result = multiply(result, _normalize(layer))
    return result

def multiply(a, b):
    size = len(a)
    out = [[0.0 for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for k in range(size):
            for j in range(size):
                out[i][j] += a[i][k] * b[k][j]
    return out

layer1 = [[0.6, 0.4, 0.0], [0.2, 0.7, 0.1], [0.1, 0.2, 0.7]]
layer2 = [[0.5, 0.3, 0.2], [0.1, 0.8, 0.1], [0.3, 0.2, 0.5]]
rollout_matrix = attention_rollout([layer1, layer2])
rollout_matrix
